# ID2214 Assignment 2 Group no. [2]
### Project members: 
[Yujie Zhang, yujiezh@kth.se]
[Muze Lu, muzel@kth.se]
[Amna Islam Badhan, badhan@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.9.7
NumPy version: 1.20.3
Pandas version: 1.3.4


## Reused functions from Assignment 1

In [15]:
def accuracy(df, correctlabels):
    N = range(len(df))
    pred = [df.iloc[i,:].idxmax() for i in N]
    indicator = [1 if pred[i] == correctlabels[i] else 0 for i in N ]
    accuracy = sum(indicator)/len(indicator)
    return accuracy

def brier_score(df, correctlabels):
    if len(df) != len(correctlabels):
        print("Error in Length")
        return None
    sum_error = 0
    for i in range(len(df)):
        f = df.iloc[i,:]
        idx = np.where(df.columns==correctlabels[i])[0]
        o = [1 if k == idx else 0 for k in range(df.shape[1])]
        sum_error += np.sum((f - o)**2)
    
    return sum_error / len(df)

def binary_auc(pred, true, col):
    
    pred = np.array(pred)
    v = [i == col for i in true] 
    col_pred, other_pred = pred[v], pred[[not i for i in v]]

    predscore = list(pred)
    predscore.extend([0,1])
    predscore = sorted(predscore)
    dictscores = {}

    for thres in predscore:
        if(len(other_pred)==0):
            FPR = 0
        else:
            FPR = sum(other_pred >= thres)/len(other_pred)
        if(len(col_pred)==0):
            TPR = 0
        else:
            TPR = sum(col_pred >= thres)/len(col_pred)
        dictscores[thres] = [FPR, TPR]

    list_reversed = [i for i in reversed(list(dictscores.values()))]

    auc = 0
    if([0,0] not in list_reversed):
        list_reversed = [[0,0]] + list_reversed
    if([1,1] not in list_reversed):
        list_reversed = list_reversed + [[1,1]]
    for i in range(len(list_reversed) - 1):
        tpr_left, tpr_right = list_reversed[i][1], list_reversed[i+1][1]
        fpr_left, fpr_right = list_reversed[i][0], list_reversed[i+1][0]
        if fpr_right != fpr_left:
            height = (tpr_left + tpr_right) / 2
            width = fpr_right - fpr_left
            auc += height*width
    return auc


def auc(df, correctlabels):
    N = len(df)
    auc = 0
    for col in df:
        weight = sum(pd.Series(correctlabels) == col)/N
        bin_auc = binary_auc(df[col], correctlabels, col)
        auc += weight*bin_auc
    return auc

def create_normalization(df, normalizationtype="minmax"):
    new_df = df.copy()
    normalization = {}
    for col in df.columns:
        if col == "CLASS" or col == "ID":
            continue
        if normalizationtype == "minmax":
            a = df[col].min()
            b = df[col].max()
            new_df[col] = (df[col] - a)/(b - a)
            normalization.update({col: (normalizationtype, a, b)})
        elif normalizationtype == "zscore":
            a = df[col].mean()
            b = df[col].std()
            new_df[col] = (df[col] - a)/b
            normalization.update({col: (normalizationtype, a, b)})
        else:
            print("There is no such type!")
        
    return new_df, normalization

def apply_normalization(df, normalization):
    new_df = df.copy()
    for col in normalization:
        normalizationtype = normalization[col][0]
        a = normalization[col][1]
        b = normalization[col][2]
        if normalizationtype == "minmax":
            new_df[col] = (df[col] - a)/(b - a)
            new_df[col] = [x if x < 1 else 1 for x in new_df[col] ] #hint 2
            new_df[col] = [x if x > 0 else 0 for x in new_df[col] ] #hint 2
        elif normalizationtype == "zscore":
            new_df[col] = (df[col] - a)/b
        else:
            print("There is no such type!")
    
    return new_df

def create_imputation(df):
    new_df = df.copy()
    imputation = {}

    for col in df.columns:
        if col not in ["ID", "CLASS"]:
            if df[col].dtype in ['float64', 'float32', 'int64', 'int32']: 
                fillvalue = df[col].mean()
                new_df[col] = new_df[col].fillna(fillvalue)
            elif df[col].dtype == "object" or df[col].dtype.name == "category": 
                fillvalue = df[col].mode()[0]
                new_df[col] = new_df[col].fillna(fillvalue)       
            imputation.update({col: fillvalue})
    return new_df, imputation

def apply_imputation(df, imputation):
    new_df = df.copy()
    for col in df.columns:
        if col not in ['ID','CLASS']:     
            fillvalue = imputation[col]
            new_df[col] = new_df[col].fillna(fillvalue)
    return new_df

def create_bins(df, nobins=10, bintype="equal-width"):
    new_df = df.copy()
    binning = {}
    for col in new_df.columns:
        if col not in ['ID', 'CLASS'] and new_df[col].dtype in ['float64', 'float32', 'int64', 'int32']:
            if bintype == 'equal-width':
                new_df[col], bins = pd.cut(new_df[col], nobins, retbins=True, labels=False)
            elif bintype == 'equal-size':
                new_df[col], bins = pd.qcut(new_df[col], nobins, retbins=True, labels=False, duplicates='drop')
            bins[0], bins[-1] = -np.inf, np.inf
            binning[col] = bins
            new_df[col] = new_df[col].astype('category')
            new_df[col] = new_df[col].cat.set_categories([str(i) for i in new_df[col].cat.categories], rename=True) #int->str
        else:
            new_df[col] = new_df[col].astype('category')
    return new_df, binning


def apply_bins(df, binning):
    new_df = df.copy()
    for col in new_df.columns:
        if(col not in ['ID', 'CLASS'] and new_df[col].dtype in ['float64', 'float32', 'int64', 'int32']):
            bins = binning[col]
            new_df[col] = pd.cut(new_df[col], bins, labels=False)
            new_df[col] = new_df[col].astype('category')
            new_df[col] = new_df[col].cat.set_categories([str(i) for i in new_df[col].cat.categories], rename = True) 
        else:
            new_df[col] = new_df[col].astype('category')
    return new_df

def create_one_hot(df):
    new_df = df.copy()
    one_hot = {}
    for col in df.columns:
        if col not in ['ID', 'CLASS']: 
            if new_df[col].dtype == 'object' or new_df[col].dtype.name ==  'category':
                new_df[col] = new_df[col].astype('category')
                one_hot[col] = new_df[col].cat.categories    
                df_one_hot = pd.get_dummies(new_df[col])
                new_df = new_df.drop(columns=col, axis=1)
                new_df = pd.concat([new_df, df_one_hot], axis=1)
                
    return new_df, one_hot

def apply_one_hot(df, one_hot):
    new_df = df.copy()
    for col in df.columns:
        if col in one_hot.keys():
            for i in one_hot[col]:
                name = col+'_'+i
                new_col = pd.Series((df[col]==i).astype('float'))
                new_df[name] = new_col
            new_df = new_df.drop(columns=col, axis=1)
    return new_df

## 1. Define the class kNN

In [110]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies



In [16]:
class kNN:
    def __init__(self):
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
    
    def fit(self, df, normalizationtype='minmax'):
        df_copy = df.copy()
        df_copy, self.imputation = create_imputation(df_copy)
        df_copy, self.normalization = create_normalization(df_copy, normalizationtype)
        df_copy, self.one_hot = create_one_hot(df_copy)
        self.training_labels = df_copy['CLASS'].astype('category')
        self.labels = self.training_labels.cat.categories
        self.training_data = df_copy.drop(columns=['CLASS', 'ID'])

    def euclidean_distance(self, x_test, x_train):
        return np.sqrt(np.sum(np.power(x_test - x_train, 2),axis=0))
    
    def get_nearest_neighbor_predictions(self, x_test, k):  
        #x_test.drop(columns=["ID","CLASS"],inplace=True)
        df = pd.DataFrame(list(np.zeros(len(self.labels)).reshape(1,6)), index=np.arange(0,x_test.shape[0]), columns=list(self.labels))
        for index, row_test in x_test.iterrows():
            distance_sort = sorted([(self.euclidean_distance(row_test.values,x_train.values),i) for i, x_train in self.training_data.iterrows()])
            distances = distance_sort[:k]
            for distance in distances:
                df.loc[index, [self.training_labels.loc[distance[1]]]] += 1
        df = df.apply(lambda x: x/k, axis=1)
        return df
    
    def predict(self, df, k=5):
        new_df = df.copy()
        new_df.drop(columns=["ID","CLASS"],inplace=True)
        
        new_df = apply_imputation(new_df,self.imputation)
        new_df = apply_normalization(new_df, self.normalization)
        new_df = apply_one_hot(new_df, self.one_hot)
        
        return self.get_nearest_neighbor_predictions(new_df, k)

In [17]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.01 s.
Testing time (k=1): 0.39 s.
Testing time (k=3): 0.51 s.
Testing time (k=5): 0.63 s.
Testing time (k=7): 0.73 s.
Testing time (k=9): 0.84 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [18]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


## 2. Define the class NaiveBayes

In [6]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#xw
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.




In [24]:
class NaiveBayes:
    def __init__(self):
        self.column_filter = None
        self.binning = None
        self.class_priors = None
        self.labels = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None

    def fit(self, df, nobins=10, bintype='equal-width'):
        df, self.column_filter = create_column_filter(df)
        df, self.binning = create_bins(df, nobins, bintype)
        df['CLASS'] = df['CLASS'].astype('category')
        self.class_priors = {c:sum(df['CLASS']==c)/len(df) for c in pd.unique(df["CLASS"])}
        self.labels = df['CLASS'].cat.categories
        self.feature_class_value_counts = {}
        self.feature_class_counts = {}
        for col in df.columns:
            if (col not in ['CLASS', 'ID']):
                self.feature_class_value_counts.update({col:(df.groupby(['CLASS', col]).size()).to_dict()})
                self.feature_class_counts.update({col:(df['CLASS'].value_counts()).to_dict()})

#    print(feature_class_counts)
#    print(feature_class_value_counts)


    def predict(self, dataframe):
        df = dataframe.copy()
        df = apply_bins(dataframe, self.binning)
    #    print(df)
        labels = list(df["CLASS"].cat.categories)
        self.labels = labels
        df.drop(columns=["CLASS", "ID"], inplace=True)
        nlabels, ncols, nrows = len(labels), len(df.columns), df.shape[0]
    #    print(nlabels, ncols, nrows)

        rel_freq = np.zeros([nlabels, ncols, nrows])
        for c in range(ncols):
            col = df.columns[c]
            for l in range(nlabels):
                label = labels[l]
                for r in range(nrows):
                    value = df.iloc[r, c]
                    t = (label, value)
                    if (t in self.feature_class_value_counts[col].keys()):
                        fvalue = self.feature_class_value_counts[col][t]
                        fcount = self.feature_class_counts[col][label]
#            print(fvalue, fcount)
                        rel_freq[l, c, r] = fvalue / fcount

        rel_freq = rel_freq.prod(axis=1)
        priors = np.array([self.class_priors[labels[i]] for i in range(nlabels)])
        priors = np.tile(priors, nrows).reshape(nrows, nlabels)
        prob = rel_freq * priors.T
        prob_sum = prob.sum(axis=0)
#    print(prob_sum.shape)
        for p in range(len(prob_sum)):
            if prob_sum[p] == 0:
                prob_sum[p] = 1
        prob = prob / prob_sum
        prediction = pd.DataFrame(prob.T, columns=labels)
    
        return prediction

In [25]:
glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")
#print(glass_test_df)
nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    nb_model.predict(glass_test_df)

In [26]:
glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")
#print(glass_test_df)
nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    nb_model.predict(glass_test_df)

In [110]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.03 s.
Testing time (3, 'equal-width'): 0.13 s.
Training time (3, 'equal-size'): 0.02 s.
Testing time (3, 'equal-size'): 0.13 s.
Training time (5, 'equal-width'): 0.02 s.
Testing time (5, 'equal-width'): 0.13 s.
Training time (5, 'equal-size'): 0.02 s.
Testing time (5, 'equal-size'): 0.13 s.
Training time (10, 'equal-width'): 0.02 s.
Testing time (10, 'equal-width'): 0.12 s.
Training time (10, 'equal-size'): 0.02 s.
Testing time (10, 'equal-size'): 0.13 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.621325  0.723560
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.626168     0.559282  0.750656
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.598131     0.570270  0.747255
   equal-size   0.579439     0.743837  0.746409

In [27]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263
